In [1]:
import sys, os

import pandas as pd

from CART import RegressionTree
from Utils.plotting import  *
from scipy.stats import norm as ndist
import joblib

# For tree-values
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector

# Select a CRAN mirror to download from
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1)  # Select the first mirror

# Install 'remotes' if it's not already installed
if not rpackages.isinstalled('remotes'):
    utils.install_packages(StrVector(('remotes',)))

import rpy2.robjects as ro

from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects import numpy2ri

In [2]:
# Run the GitHub installation command for 'treevalues'
ro.r('remotes::install_github("anna-neufeld/treevalues")')
ro.r('library(treevalues)')
ro.r('library(rpart)')

R[write to console]: Using GitHub PAT from the git credential store.

R[write to console]: Skipping install of 'treevalues' from a github remote, the SHA1 (55573782) has not changed since last install.
  Use `force = TRUE` to force installation



In [3]:
def generate_test(mu, sd_y):
    n = mu.shape[0]
    return mu + np.random.normal(size=(n,), scale=sd_y)

# Tree-values inference

In [4]:
def tree_values_inference(X, y, sd_y, mu, max_depth=5, level=0.1,
                          X_test=None):
    # Convert the NumPy matrix to an R matrix
    X_r = numpy2ri.py2rpy(X)
    y_r = numpy2ri.py2rpy(y)

    # Assign the R matrix to a variable in the R environment (optional)
    ro.globalenv['X_r'] = X_r
    ro.globalenv['y_r'] = y_r
    ro.globalenv['p'] = X.shape[1]

    # Construct dataset
    ro.r('data <- cbind(y_r, X_r)')
    # Set the column names to "y", "x1", "x2", ..., "x10"
    ro.r('colnames(data) <- c("y", paste0("x", 1:p))')
    ro.r('data = data.frame(data)')

    # Define the rpart tree model
    tree_cmd = ('bls.tree <- rpart(y ~ ., data=data, model = TRUE, ' +
                'control = rpart.control(cp=0.00, minsplit = 25, minbucket = 10, maxdepth=') + str(max_depth) + '))'
    ro.r(tree_cmd)
    bls_tree = ro.r('bls.tree')
    # Plot the tree values (this will plot directly if you have a plotting backend set up)
    # ro.r('treeval.plot(bls.tree, inferenceType=0)')

    # ro.r('print(row.names(bls.tree$frame)[bls.tree$frame$var == "<leaf>"])')
    ro.r('leaf_idx <- (row.names(bls.tree$frame)[bls.tree$frame$var == "<leaf>"])')
    leaf_idx = ro.r['leaf_idx']

    # Get node mapping
    ro.r('idx_full <- 1:nrow(bls.tree$frame)')
    ro.r('mapped_idx <- idx_full[bls.tree$frame$var == "<leaf>"]')

    len = []
    coverage = []

    for i, idx in enumerate(leaf_idx):
        # Get the branch information for a specific branch in the tree
        command = 'branch <- getBranch(bls.tree, ' + str(idx) + ')'
        ro.r(command)
        # Perform branch inference
        ro.r(f'result <- branchInference(bls.tree, branch, type="reg", alpha = 0.10, sigma_y={sd_y})')
        # Get confidence intervals
        confint = ro.r('result$confint')
        len.append(confint[1] - confint[0])

        target_cmd = "contrast <- (bls.tree$where == mapped_idx[" + str(i + 1) + "])"
        ro.r(target_cmd)
        contrast = ro.r('contrast')
        contrast = np.array(contrast)

        contrast = np.array(contrast * 1 / np.sum(contrast))

        target = contrast.dot(mu)
        coverage.append(target >= confint[0] and target <= confint[1])

    if X_test is not None:
        X_test_r = numpy2ri.py2rpy(X_test)
        ro.globalenv['X_test_r'] = X_test_r
        ro.r('pred <- predict(bls.tree, data = X_test_r)')
        pred = ro.r['pred']
    else:
        pred = None

    return (np.mean(coverage), np.mean(len), pred)


# Inference with UV decomposition

In [5]:
def UV_decomposition(X, y, mu, sd_y,
                     max_depth=5, min_prop=0, min_sample=10, min_bucket=5,
                     level=0.1, gamma=1,
                     X_test=None):
    n = X.shape[0]
    W = np.random.normal(loc=0, scale=sd_y * np.sqrt(gamma), size=(n,))
    U = y + W
    V = y - W / gamma
    sd_V = sd_y * np.sqrt(1 + 1 / gamma)
    reg_tree = RegressionTree(min_samples_split=min_sample, max_depth=max_depth,
                              min_proportion=min_prop, min_bucket=min_bucket)
    reg_tree.fit(X, U, sd=0)

    coverage = []
    lengths = []

    for node in reg_tree.terminal_nodes:
        contrast = node.membership

        contrast = np.array(contrast * 1 / np.sum(contrast))

        target = contrast.dot(mu)

        # Naive after tree value
        # Confidence intervals
        CI = [contrast.dot(V) -
              np.linalg.norm(contrast) * sd_V * ndist.ppf(1 - level / 2),
              contrast.dot(V) +
              np.linalg.norm(contrast) * sd_V * ndist.ppf(1 - level / 2)]
        coverage.append((target >= CI[0] and target <= CI[1]))
        lengths.append(CI[1] - CI[0])

    if X_test is not None:
        pred = reg_tree.predict(X_test)
    else:
        pred = None

    return coverage, lengths, pred

# RRT inference

In [6]:
def randomized_inference(reg_tree, sd_y, y, mu, level=0.1):
    # print(reg_tree.terminal_nodes)
    coverage_i = []
    lengths_i = []

    for node in reg_tree.terminal_nodes:
        pval, dist, contrast, norm_contrast, obs_tar, logW, suff, sel_probs \
            = (reg_tree.condl_node_inference(node=node,
                                             #ngrid=10000,
                                             #ncoarse=300,
                                             ngrid=10000,
                                             ncoarse=50,
                                             grid_w_const=5,
                                             reduced_dim=1,
                                             sd=sd_y,
                                             use_cvxpy=True))
        target = contrast.dot(mu)

        # This is an interval for
        # eta_*'mu = eta'mu / (norm(eta) * sd_y)
        selective_CI = (dist.equal_tailed_interval(observed=norm_contrast.dot(y),
                                                   alpha=level))
        selective_CI = np.array(selective_CI)
        selective_CI *= np.linalg.norm(contrast) * sd_y
        coverage_i.append((target >= selective_CI[0] and target <= selective_CI[1]))
        lengths_i.append(selective_CI[1] - selective_CI[0])

    return coverage_i, lengths_i

# Replicating Figure 4

In [10]:
def vary_p_sim(n=50, p_list=[5, 20, 50], sd_y=5, noise_sd=1,
               start=0, end=100, level=0.1, path=None):
    oper_char = {}
    oper_char["Coverage Rate"] = []
    oper_char["Length"] = []
    oper_char["MSE"] = []
    oper_char["Method"] = []
    oper_char["p"] = []
    # oper_char["a"] = []
    # oper_char["b"] = []
    a = 1
    b = 2

    # for ab_prod in itertools.product(a_list, b_list):
    # a = ab_prod[0]
    # b = ab_prod[1]
    for i in range(start, end):
        for p in p_list:
            print(i, "th simulation")
            np.random.seed(i + 10000)
            X = np.random.normal(size=(n, p))

            mu = b * ((X[:, 0] <= 0) * (1 + a * (X[:, 1] > 0) + (X[:, 2] * X[:, 1] <= 0)))
            y = mu + np.random.normal(size=(n,), scale=sd_y)
            y_test = generate_test(mu, sd_y)

            # Create and train the regression tree
            reg_tree = RegressionTree(min_samples_split=50, max_depth=3,
                                      min_proportion=0., min_bucket=20)
            reg_tree.fit(X, y, sd=noise_sd * sd_y)

            # RRT Inference
            coverage_i, lengths_i = randomized_inference(reg_tree=reg_tree,
                                                         y=y, sd_y=sd_y, mu=mu,
                                                         level=level)
            pred_test = reg_tree.predict(X)
            MSE_test = (np.mean((y_test - pred_test) ** 2))
            # Record results
            oper_char["Coverage Rate"].append(np.mean(coverage_i))
            oper_char["Length"].append(np.mean(lengths_i))
            oper_char["MSE"].append(MSE_test)
            oper_char["Method"].append(f"RRT({noise_sd})")
            oper_char["p"].append(p)
            # oper_char["a"].append(a)
            # oper_char["b"].append(b)

            # Tree value & naive inference & prediction
            (coverage_treeval, avg_len_treeval,
             pred_test_treeval) = tree_values_inference(X, y, sd_y, mu,
                                                        X_test=X, max_depth=3)
            MSE_test_treeval = (np.mean((y_test - pred_test_treeval) ** 2))

            oper_char["Coverage Rate"].append(coverage_treeval)
            oper_char["Length"].append(avg_len_treeval)
            oper_char["MSE"].append(MSE_test_treeval)
            oper_char["Method"].append("Tree-Values")
            oper_char["p"].append(p)

            # UV decomposition
            coverage_UV, len_UV, pred_UV = UV_decomposition(X, y, mu, sd_y, X_test=X,
                                                            min_prop=0., max_depth=3,
                                                            min_sample=50, min_bucket=20,
                                                            gamma=0.1)

            MSE_test_UV = (np.mean((y_test - pred_UV) ** 2))

            oper_char["Coverage Rate"].append(np.mean(coverage_UV))
            oper_char["Length"].append(np.mean(len_UV))
            oper_char["MSE"].append(MSE_test_UV)
            oper_char["Method"].append("UV(0.1)")
            oper_char["p"].append(p)

        if path is not None:
            joblib.dump(oper_char, path)

    return oper_char

In [11]:
oper_char = vary_p_sim(n=200, p_list=[5, 10, 20], sd_y=5, noise_sd=1,
                       start=0, end=3, level=0.1, path=None)

0 th simulation
0 th simulation
0 th simulation
1 th simulation
1 th simulation
1 th simulation
2 th simulation
2 th simulation
2 th simulation


In [12]:
# Row: each row is one simulation for one method under one particular value of p
pd.DataFrame(oper_char)

,Coverage Rate,Length,MSE,Method,p
0,1.000000,5.862767,31.427251,RRT(1),5
1,0.666667,74.201922,32.484935,Tree-Values,5
2,1.000000,10.006228,30.793123,UV(0.1),5
3,0.833333,8.957472,35.264500,RRT(1),10
4,0.875000,47.698143,35.180564,Tree-Values,10
5,0.833333,9.611967,35.287628,UV(0.1),10
6,0.600000,3.541435,31.326035,RRT(1),20
7,1.000000,38.536912,32.949922,Tree-Values,20
8,1.000000,9.664580,31.143904,UV(0.1),20
9,0.666667,6.799782,27.295528,RRT(1),5
